# 📚 DAY 2 - NOTEBOOK 1: READING DATA

## 🎯 Objectives:
- Read data from multiple formats (CSV, JSON, Parquet)
- Understand schema inference vs explicit schema
- Use read options effectively
- Handle corrupted/malformed data
- Compare performance of different formats
- Work with both local filesystem and MinIO (S3)

## 📂 Data Pipeline:
```
raw/ → staging/ → production/
```

---
## 🔧 PART 1: SETUP SPARK SESSION

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when, expr, count, isnan, isnull
from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType, 
    DoubleType, DateType, TimestampType
)
import time
import os

print("📦 Importing libraries...")
print("✅ All imports successful!")

📦 Importing libraries...
✅ All imports successful!


In [2]:
# Stop existing Spark session if any
try:
    spark.stop()
    print("🛑 Stopped existing Spark session")
    time.sleep(2)
except:
    print("ℹ️  No existing Spark session to stop")
    pass

ℹ️  No existing Spark session to stop


In [3]:
print("🚀 Creating Spark Session with MinIO support...")
print("="*70)

spark = SparkSession.builder \
    .appName("Day2-ReadingData") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "512m") \
    .config("spark.executor.cores", "1") \
    .config("spark.cores.max", "2") \
    .config("spark.sql.shuffle.partitions", "4") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

print("✅ Spark Session Created Successfully!")
print(f"   App Name: {spark.sparkContext.appName}")
print(f"   App ID: {spark.sparkContext.applicationId}")
print(f"   Master: {spark.sparkContext.master}")
print(f"   Spark Version: {spark.version}")
print(f"   Python Version: {spark.sparkContext.pythonVer}")

🚀 Creating Spark Session with MinIO support...


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/03 10:35:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✅ Spark Session Created Successfully!
   App Name: Day2-ReadingData
   App ID: app-20260103103547-0000
   Master: spark://spark-master:7077
   Spark Version: 3.5.1
   Python Version: 3.8


---
## 📂 PART 2: DEFINE DATA PATHS

In [4]:
# ============================================================================
# DEFINE DATA PATHS
# ============================================================================

# Option 1: Local filesystem (shared volume)
DATA_RAW_LOCAL = "/opt/spark-data/raw"
DATA_STAGING_LOCAL = "/opt/spark-data/staging"
DATA_PRODUCTION_LOCAL = "/opt/spark-data/production"

# Option 2: MinIO (S3-compatible)
DATA_RAW_S3 = "s3a://raw"
DATA_STAGING_S3 = "s3a://staging"
DATA_PRODUCTION_S3 = "s3a://production"

print("📂 Available data paths:")
print("="*70)
print("\n🗂️  LOCAL FILESYSTEM (Shared Volume):")
print(f"   Raw:        {DATA_RAW_LOCAL}")
print(f"   Staging:    {DATA_STAGING_LOCAL}")
print(f"   Production: {DATA_PRODUCTION_LOCAL}")

print("\n☁️  MINIO (S3-Compatible):")
print(f"   Raw:        {DATA_RAW_S3}")
print(f"   Staging:    {DATA_STAGING_S3}")
print(f"   Production: {DATA_PRODUCTION_S3}")

print("\n💡 TIP: Use local filesystem for development, MinIO for production")

📂 Available data paths:

🗂️  LOCAL FILESYSTEM (Shared Volume):
   Raw:        /opt/spark-data/raw
   Staging:    /opt/spark-data/staging
   Production: /opt/spark-data/production

☁️  MINIO (S3-Compatible):
   Raw:        s3a://raw
   Staging:    s3a://staging
   Production: s3a://production

💡 TIP: Use local filesystem for development, MinIO for production


---
## 📝 PART 3: CREATE SAMPLE DATA

In [5]:
from faker import Faker
import random
import pandas as pd

# Set seeds for reproducibility
fake = Faker()
Faker.seed(42)
random.seed(42)

print("📝 Creating sample employee data...")
print("="*70)

# Create employees data
employees_data = []
departments = ["Engineering", "Sales", "HR", "Marketing", "Finance"]

for i in range(1, 1001):
    employees_data.append({
        "id": i,
        "name": fake.name(),
        "email": fake.email(),
        "department": random.choice(departments),
        "salary": random.randint(40000, 150000),
        "age": random.randint(22, 65),
        "hire_date": fake.date_between(start_date='-10y', end_date='today').strftime('%Y-%m-%d')
    })

print(f"✅ Created {len(employees_data)} employee records")
print(f"   Departments: {', '.join(departments)}")
print(f"   Salary range: $40,000 - $150,000")
print(f"   Age range: 22 - 65")
print(f"   Hire date range: Last 10 years")

📝 Creating sample employee data...
✅ Created 1000 employee records
   Departments: Engineering, Sales, HR, Marketing, Finance
   Salary range: $40,000 - $150,000
   Age range: 22 - 65
   Hire date range: Last 10 years


In [6]:
# Preview sample data
print("\n📊 Sample data preview:")
print("="*70)
df_preview = pd.DataFrame(employees_data[:5])
print(df_preview.to_string(index=False))


📊 Sample data preview:
 id              name                    email  department  salary  age  hire_date
  1      Allison Hill donaldgarcia@example.net Engineering   43278   39 2024-12-05
  2    Javier Johnson  jesseguzman@example.net       Sales   69256   30 2017-12-29
  3 Kimberly Robinson       lisa02@example.net Engineering  128696   56 2020-03-25
  4  Daniel Gallagher   daviscolin@example.com Engineering  117397   49 2022-01-16
  5    Monica Herrera      smiller@example.net Engineering   43905   27 2022-03-11


In [7]:
# Create directory structure
print("\n📁 Creating directory structure...")
print("="*70)

os.makedirs(f"{DATA_RAW_LOCAL}/employees", exist_ok=True)
print(f"✅ Created: {DATA_RAW_LOCAL}/employees/")

# Save to different formats
print("\n💾 Saving data to multiple formats...")

# 1. CSV
df_pandas = pd.DataFrame(employees_data)
csv_path = f'{DATA_RAW_LOCAL}/employees/employees.csv'
df_pandas.to_csv(csv_path, index=False)
csv_size = os.path.getsize(csv_path) / 1024  # KB
print(f"✅ CSV saved: {csv_path}")
print(f"   Size: {csv_size:.2f} KB")

# 2. JSON (line-delimited)
json_path = f'{DATA_RAW_LOCAL}/employees/employees.json'
df_pandas.to_json(json_path, orient='records', lines=True)
json_size = os.path.getsize(json_path) / 1024  # KB
print(f"✅ JSON saved: {json_path}")
print(f"   Size: {json_size:.2f} KB")

# 3. Parquet (using Spark)
df_spark = spark.createDataFrame(employees_data)
parquet_path = f'{DATA_RAW_LOCAL}/employees/employees.parquet'
df_spark.write.mode('overwrite').parquet(parquet_path)
print(f"✅ Parquet saved: {parquet_path}")

print("\n📊 File size comparison:")
print(f"   CSV:     {csv_size:.2f} KB")
print(f"   JSON:    {json_size:.2f} KB")
print(f"   Parquet: (columnar format - typically smallest)")


📁 Creating directory structure...
✅ Created: /opt/spark-data/raw/employees/

💾 Saving data to multiple formats...
✅ CSV saved: /opt/spark-data/raw/employees/employees.csv
   Size: 67.83 KB
✅ JSON saved: /opt/spark-data/raw/employees/employees.json
   Size: 136.14 KB


✅ Parquet saved: /opt/spark-data/raw/employees/employees.parquet

📊 File size comparison:
   CSV:     67.83 KB
   JSON:    136.14 KB
   Parquet: (columnar format - typically smallest)


---
## 📖 PART 4: READING CSV FILES

In [8]:
print("📌 READING CSV - Basic (with schema inference)")
print("="*70)

# Read CSV with schema inference
start_time = time.time()

df_csv = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(f"{DATA_RAW_LOCAL}/employees/employees.csv")

read_time = time.time() - start_time

print("✅ CSV loaded successfully")
print(f"   Rows: {df_csv.count()}")
print(f"   Columns: {len(df_csv.columns)}")
print(f"   Read time: {read_time:.3f} seconds")

print("\n📋 Inferred Schema:")
df_csv.printSchema()

print("\n📊 Sample data:")
df_csv.show(5, truncate=False)

📌 READING CSV - Basic (with schema inference)


✅ CSV loaded successfully
   Rows: 1000
   Columns: 7
   Read time: 4.460 seconds

📋 Inferred Schema:
root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- hire_date: date (nullable = true)


📊 Sample data:
+---+-----------------+------------------------+-----------+------+---+----------+
|id |name             |email                   |department |salary|age|hire_date |
+---+-----------------+------------------------+-----------+------+---+----------+
|1  |Allison Hill     |donaldgarcia@example.net|Engineering|43278 |39 |2024-12-05|
|2  |Javier Johnson   |jesseguzman@example.net |Sales      |69256 |30 |2017-12-29|
|3  |Kimberly Robinson|lisa02@example.net      |Engineering|128696|56 |2020-03-25|
|4  |Daniel Gallagher |daviscolin@example.com  |Engineering|117397|49 |2022-01-16|
|5  |Monica Herrera   |smiller@

In [9]:
print("📌 READING CSV - With Explicit Schema (RECOMMENDED)")
print("="*70)

# Define explicit schema
employee_schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("name", StringType(), False),
    StructField("email", StringType(), True),
    StructField("department", StringType(), False),
    StructField("salary", IntegerType(), False),
    StructField("age", IntegerType(), False),
    StructField("hire_date", DateType(), False)
])

print("📋 Defined Schema:")
print(employee_schema.simpleString())

# Read with explicit schema
start_time = time.time()

df_csv_explicit = spark.read \
    .option("header", "true") \
    .option("dateFormat", "yyyy-MM-dd") \
    .schema(employee_schema) \
    .csv(f"{DATA_RAW_LOCAL}/employees/employees.csv")

read_time = time.time() - start_time

print("\n✅ CSV loaded with explicit schema")
print(f"   Read time: {read_time:.3f} seconds")
print("   ⚡ Faster than schema inference!")

print("\n📋 Schema:")
df_csv_explicit.printSchema()

print("\n📊 Sample data:")
df_csv_explicit.show(5)

📌 READING CSV - With Explicit Schema (RECOMMENDED)
📋 Defined Schema:
struct<id:int,name:string,email:string,department:string,salary:int,age:int,hire_date:date>

✅ CSV loaded with explicit schema
   Read time: 0.068 seconds
   ⚡ Faster than schema inference!

📋 Schema:
root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- hire_date: date (nullable = true)


📊 Sample data:
+---+-----------------+--------------------+-----------+------+---+----------+
| id|             name|               email| department|salary|age| hire_date|
+---+-----------------+--------------------+-----------+------+---+----------+
|  1|     Allison Hill|donaldgarcia@exam...|Engineering| 43278| 39|2024-12-05|
|  2|   Javier Johnson|jesseguzman@examp...|      Sales| 69256| 30|2017-12-29|
|  3|Kimberly Robinson|  lisa02@example.net|Engin

In [10]:
print("📌 CSV READ OPTIONS - Advanced")
print("="*70)

# Common CSV options
df_csv_advanced = spark.read \
    .option("header", "true") \
    .option("inferSchema", "false") \
    .option("sep", ",") \
    .option("quote", '"') \
    .option("escape", "\\") \
    .option("nullValue", "NULL") \
    .option("dateFormat", "yyyy-MM-dd") \
    .option("timestampFormat", "yyyy-MM-dd HH:mm:ss") \
    .option("mode", "PERMISSIVE") \
    .option("columnNameOfCorruptRecord", "_corrupt_record") \
    .schema(employee_schema) \
    .csv(f"{DATA_RAW_LOCAL}/employees/employees.csv")

print("✅ CSV loaded with advanced options")
print("\n📝 Options explained:")
print("   • header: First row is column names")
print("   • sep: Field delimiter (comma)")
print("   • quote: Quote character for strings")
print("   • escape: Escape character")
print("   • nullValue: String to treat as NULL")
print("   • dateFormat: Date parsing format")
print("   • mode: How to handle corrupt records")
print("   • columnNameOfCorruptRecord: Column for bad records")

df_csv_advanced.show(5)

📌 CSV READ OPTIONS - Advanced
✅ CSV loaded with advanced options

📝 Options explained:
   • header: First row is column names
   • sep: Field delimiter (comma)
   • quote: Quote character for strings
   • escape: Escape character
   • nullValue: String to treat as NULL
   • dateFormat: Date parsing format
   • mode: How to handle corrupt records
   • columnNameOfCorruptRecord: Column for bad records
+---+-----------------+--------------------+-----------+------+---+----------+
| id|             name|               email| department|salary|age| hire_date|
+---+-----------------+--------------------+-----------+------+---+----------+
|  1|     Allison Hill|donaldgarcia@exam...|Engineering| 43278| 39|2024-12-05|
|  2|   Javier Johnson|jesseguzman@examp...|      Sales| 69256| 30|2017-12-29|
|  3|Kimberly Robinson|  lisa02@example.net|Engineering|128696| 56|2020-03-25|
|  4| Daniel Gallagher|daviscolin@exampl...|Engineering|117397| 49|2022-01-16|
|  5|   Monica Herrera| smiller@example.net|

---
## 📖 PART 5: READING JSON FILES

In [11]:
print("📌 READING JSON - Line-delimited (JSONL)")
print("="*70)

start_time = time.time()

df_json = spark.read.json(f"{DATA_RAW_LOCAL}/employees/employees.json")

read_time = time.time() - start_time

print("✅ JSON loaded successfully")
print(f"   Rows: {df_json.count()}")
print(f"   Columns: {len(df_json.columns)}")
print(f"   Read time: {read_time:.3f} seconds")

print("\n📋 Schema (auto-inferred):")
df_json.printSchema()

print("\n📊 Sample data:")
df_json.show(5)

📌 READING JSON - Line-delimited (JSONL)
✅ JSON loaded successfully
   Rows: 1000
   Columns: 7
   Read time: 0.403 seconds

📋 Schema (auto-inferred):
root
 |-- age: long (nullable = true)
 |-- department: string (nullable = true)
 |-- email: string (nullable = true)
 |-- hire_date: string (nullable = true)
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: long (nullable = true)


📊 Sample data:
+---+-----------+--------------------+----------+---+-----------------+------+
|age| department|               email| hire_date| id|             name|salary|
+---+-----------+--------------------+----------+---+-----------------+------+
| 39|Engineering|donaldgarcia@exam...|2024-12-05|  1|     Allison Hill| 43278|
| 30|      Sales|jesseguzman@examp...|2017-12-29|  2|   Javier Johnson| 69256|
| 56|Engineering|  lisa02@example.net|2020-03-25|  3|Kimberly Robinson|128696|
| 49|Engineering|daviscolin@exampl...|2022-01-16|  4| Daniel Gallagher|117397|
| 27|Engineering| 

In [12]:
print("📌 READING JSON - With Explicit Schema")
print("="*70)

start_time = time.time()

df_json_explicit = spark.read \
    .schema(employee_schema) \
    .json(f"{DATA_RAW_LOCAL}/employees/employees.json")

read_time = time.time() - start_time

print("✅ JSON loaded with explicit schema")
print(f"   Read time: {read_time:.3f} seconds")
print("   ⚡ Faster than schema inference!")

df_json_explicit.show(5)

📌 READING JSON - With Explicit Schema
✅ JSON loaded with explicit schema
   Read time: 0.051 seconds
   ⚡ Faster than schema inference!
+---+-----------------+--------------------+-----------+------+---+----------+
| id|             name|               email| department|salary|age| hire_date|
+---+-----------------+--------------------+-----------+------+---+----------+
|  1|     Allison Hill|donaldgarcia@exam...|Engineering| 43278| 39|2024-12-05|
|  2|   Javier Johnson|jesseguzman@examp...|      Sales| 69256| 30|2017-12-29|
|  3|Kimberly Robinson|  lisa02@example.net|Engineering|128696| 56|2020-03-25|
|  4| Daniel Gallagher|daviscolin@exampl...|Engineering|117397| 49|2022-01-16|
|  5|   Monica Herrera| smiller@example.net|Engineering| 43905| 27|2022-03-11|
+---+-----------------+--------------------+-----------+------+---+----------+
only showing top 5 rows



---
## 📖 PART 6: READING PARQUET FILES

In [13]:
print("📌 READING PARQUET - Columnar Format")
print("="*70)

start_time = time.time()

df_parquet = spark.read.parquet(f"{DATA_RAW_LOCAL}/employees/employees.parquet")

read_time = time.time() - start_time

print("✅ Parquet loaded successfully")
print(f"   Rows: {df_parquet.count()}")
print(f"   Columns: {len(df_parquet.columns)}")
print(f"   Read time: {read_time:.3f} seconds")
print("   ⚡ Fastest format!")

print("\n📋 Schema (embedded in Parquet):")
df_parquet.printSchema()

print("\n📊 Sample data:")
df_parquet.show(5)

print("\n💡 Parquet advantages:")
print("   • Schema is embedded (no inference needed)")
print("   • Columnar storage (efficient for analytics)")
print("   • Compressed by default")
print("   • Supports predicate pushdown")
print("   • Best for production workloads")

📌 READING PARQUET - Columnar Format
✅ Parquet loaded successfully
   Rows: 1000
   Columns: 7
   Read time: 0.314 seconds
   ⚡ Fastest format!

📋 Schema (embedded in Parquet):
root
 |-- age: long (nullable = true)
 |-- department: string (nullable = true)
 |-- email: string (nullable = true)
 |-- hire_date: string (nullable = true)
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: long (nullable = true)


📊 Sample data:
+---+-----------+--------------------+----------+---+-----------------+------+
|age| department|               email| hire_date| id|             name|salary|
+---+-----------+--------------------+----------+---+-----------------+------+
| 39|Engineering|donaldgarcia@exam...|2024-12-05|  1|     Allison Hill| 43278|
| 30|      Sales|jesseguzman@examp...|2017-12-29|  2|   Javier Johnson| 69256|
| 56|Engineering|  lisa02@example.net|2020-03-25|  3|Kimberly Robinson|128696|
| 49|Engineering|daviscolin@exampl...|2022-01-16|  4| Daniel Gallagher|

---
## 📊 PART 7: PERFORMANCE COMPARISON

In [14]:
print("📊 PERFORMANCE COMPARISON")
print("="*70)

import time

results = []

# Test CSV
print("\n🔄 Testing CSV...")
start = time.time()
df_csv_test = spark.read.option("header", "true").option("inferSchema", "true").csv(f"{DATA_RAW_LOCAL}/employees/employees.csv")
count_csv = df_csv_test.count()
time_csv = time.time() - start
results.append(("CSV (inferred)", time_csv, count_csv))
print(f"   Time: {time_csv:.3f}s")

# Test CSV with explicit schema
print("\n🔄 Testing CSV (explicit schema)...")
start = time.time()
df_csv_explicit_test = spark.read.option("header", "true").schema(employee_schema).csv(f"{DATA_RAW_LOCAL}/employees/employees.csv")
count_csv_explicit = df_csv_explicit_test.count()
time_csv_explicit = time.time() - start
results.append(("CSV (explicit)", time_csv_explicit, count_csv_explicit))
print(f"   Time: {time_csv_explicit:.3f}s")

# Test JSON
print("\n🔄 Testing JSON...")
start = time.time()
df_json_test = spark.read.json(f"{DATA_RAW_LOCAL}/employees/employees.json")
count_json = df_json_test.count()
time_json = time.time() - start
results.append(("JSON (inferred)", time_json, count_json))
print(f"   Time: {time_json:.3f}s")

# Test Parquet
print("\n🔄 Testing Parquet...")
start = time.time()
df_parquet_test = spark.read.parquet(f"{DATA_RAW_LOCAL}/employees/employees.parquet")
count_parquet = df_parquet_test.count()
time_parquet = time.time() - start
results.append(("Parquet", time_parquet, count_parquet))
print(f"   Time: {time_parquet:.3f}s")

# Display results
print("\n" + "="*70)
print("📊 RESULTS SUMMARY")
print("="*70)
print(f"{'Format':<20} {'Time (s)':<15} {'Rows':<10} {'Speedup'}")
print("-"*70)

baseline = time_csv
for format_name, time_taken, row_count in results:
    speedup = baseline / time_taken
    print(f"{format_name:<20} {time_taken:<15.3f} {row_count:<10} {speedup:.2f}x")

print("\n🏆 Winner: Parquet (fastest)")
print("💡 Recommendation: Use Parquet for production workloads")

📊 PERFORMANCE COMPARISON

🔄 Testing CSV...
   Time: 4.224s

🔄 Testing CSV (explicit schema)...
   Time: 0.314s

🔄 Testing JSON...
   Time: 0.551s

🔄 Testing Parquet...
   Time: 0.574s

📊 RESULTS SUMMARY
Format               Time (s)        Rows       Speedup
----------------------------------------------------------------------
CSV (inferred)       4.224           1000       1.00x
CSV (explicit)       0.314           1000       13.46x
JSON (inferred)      0.551           1000       7.67x
Parquet              0.574           1000       7.36x

🏆 Winner: Parquet (fastest)
💡 Recommendation: Use Parquet for production workloads


---
## 🛡️ PART 8: HANDLING CORRUPTED DATA

In [15]:
print("📝 Creating corrupted CSV file for testing...")
print("="*70)

# Create a CSV with some corrupted records
corrupted_csv = f"{DATA_RAW_LOCAL}/employees/employees_corrupted.csv"

with open(corrupted_csv, 'w') as f:
    f.write("id,name,email,department,salary,age,hire_date\n")
    f.write("1,John Doe,john@example.com,Engineering,75000,30,2020-01-15\n")
    f.write("2,Jane Smith,jane@example.com,Sales,65000,28,2021-03-20\n")
    f.write("3,Bad Record,missing_fields\n")  # ❌ Corrupted: missing fields
    f.write("4,Bob Johnson,bob@example.com,HR,55000,35,2019-06-10\n")
    f.write("5,Alice,alice@example.com,Marketing,INVALID,29,2022-01-05\n")  # ❌ Corrupted: invalid salary
    f.write("6,Charlie Brown,charlie@example.com,Finance,70000,32,2020-09-15\n")

print(f"✅ Created corrupted CSV: {corrupted_csv}")
print("   Contains 2 corrupted records out of 6 total")

📝 Creating corrupted CSV file for testing...
✅ Created corrupted CSV: /opt/spark-data/raw/employees/employees_corrupted.csv
   Contains 2 corrupted records out of 6 total


In [16]:
print("📌 MODE: PERMISSIVE (default) - Keep corrupted records")
print("="*70)

df_permissive = spark.read \
    .option("header", "true") \
    .option("mode", "PERMISSIVE") \
    .option("columnNameOfCorruptRecord", "_corrupt_record") \
    .schema(employee_schema.add("_corrupt_record", StringType(), True)) \
    .csv(corrupted_csv)

print("✅ Data loaded with PERMISSIVE mode")
print(f"   Total rows: {df_permissive.count()}")

print("\n📊 All records (including corrupted):")
df_permissive.show(10, truncate=False)

print("\n🔍 Corrupted records only:")
df_corrupted = df_permissive.filter(col("_corrupt_record").isNotNull())
print(f"   Found {df_corrupted.count()} corrupted records")
df_corrupted.show(truncate=False)

📌 MODE: PERMISSIVE (default) - Keep corrupted records
✅ Data loaded with PERMISSIVE mode
   Total rows: 6

📊 All records (including corrupted):
+---+-------------+-------------------+-----------+------+----+----------+---------------------------------------------------------+
|id |name         |email              |department |salary|age |hire_date |_corrupt_record                                          |
+---+-------------+-------------------+-----------+------+----+----------+---------------------------------------------------------+
|1  |John Doe     |john@example.com   |Engineering|75000 |30  |2020-01-15|NULL                                                     |
|2  |Jane Smith   |jane@example.com   |Sales      |65000 |28  |2021-03-20|NULL                                                     |
|3  |Bad Record   |missing_fields     |NULL       |NULL  |NULL|NULL      |3,Bad Record,missing_fields                              |
|4  |Bob Johnson  |bob@example.com    |HR         |55000 |

In [17]:
print("📌 MODE: DROPMALFORMED - Drop corrupted records")
print("="*70)

df_dropmalformed = spark.read \
    .option("header", "true") \
    .option("mode", "DROPMALFORMED") \
    .schema(employee_schema) \
    .csv(corrupted_csv)

print("✅ Data loaded with DROPMALFORMED mode")
print(f"   Total rows: {df_dropmalformed.count()}")
print("   ⚠️  Corrupted records were dropped")

print("\n📊 Valid records only:")
df_dropmalformed.show(truncate=False)

📌 MODE: DROPMALFORMED - Drop corrupted records
✅ Data loaded with DROPMALFORMED mode
   Total rows: 6
   ⚠️  Corrupted records were dropped

📊 Valid records only:
+---+-------------+-------------------+-----------+------+---+----------+---------------+
|id |name         |email              |department |salary|age|hire_date |_corrupt_record|
+---+-------------+-------------------+-----------+------+---+----------+---------------+
|1  |John Doe     |john@example.com   |Engineering|75000 |30 |2020-01-15|NULL           |
|2  |Jane Smith   |jane@example.com   |Sales      |65000 |28 |2021-03-20|NULL           |
|4  |Bob Johnson  |bob@example.com    |HR         |55000 |35 |2019-06-10|NULL           |
|6  |Charlie Brown|charlie@example.com|Finance    |70000 |32 |2020-09-15|NULL           |
+---+-------------+-------------------+-----------+------+---+----------+---------------+



In [18]:
print("📌 MODE: FAILFAST - Fail on corrupted records")
print("="*70)

try:
    df_failfast = spark.read \
        .option("header", "true") \
        .option("mode", "FAILFAST") \
        .schema(employee_schema) \
        .csv(corrupted_csv)
    
    # This will trigger the error
    df_failfast.show()
    
except Exception as e:
    print("❌ FAILFAST mode detected corrupted data!")
    print(f"   Error: {str(e)[:200]}...")
    print("\n💡 Use FAILFAST in production to catch data quality issues early")

📌 MODE: FAILFAST - Fail on corrupted records


26/01/03 10:36:16 WARN TaskSetManager: Lost task 0.0 in stage 48.0 (TID 41) (172.18.0.7 executor 1): org.apache.spark.SparkException: [MALFORMED_RECORD_IN_PARSING.WITHOUT_SUGGESTION] Malformed records are detected in record parsing: [3,Bad Record,missing_fields,null,null,null,null,3,Bad Record,missing_fields].
Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'. 
	at org.apache.spark.sql.errors.QueryExecutionErrors$.malformedRecordsDetectedInRecordParsingError(QueryExecutionErrors.scala:1610)
	at org.apache.spark.sql.catalyst.util.FailureSafeParser.parse(FailureSafeParser.scala:79)
	at org.apache.spark.sql.catalyst.csv.UnivocityParser$.$anonfun$parseIterator$2(UnivocityParser.scala:456)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.execution.datasou

❌ FAILFAST mode detected corrupted data!
   Error: An error occurred while calling o199.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 48.0 failed 4 times, most recent failure: Lost task 0.3 in stage 4...

💡 Use FAILFAST in production to catch data quality issues early


---
## 📤 PART 9: READING MULTIPLE FILES

In [19]:
print("📝 Creating multiple CSV files...")
print("="*70)

# Create directory for multiple files
multi_dir = f"{DATA_RAW_LOCAL}/employees_multi"
os.makedirs(multi_dir, exist_ok=True)

# Split data into 3 files
chunk_size = len(employees_data) // 3

for i in range(3):
    start_idx = i * chunk_size
    end_idx = start_idx + chunk_size if i < 2 else len(employees_data)
    chunk_data = employees_data[start_idx:end_idx]
    
    df_chunk = pd.DataFrame(chunk_data)
    df_chunk.to_csv(f"{multi_dir}/employees_part_{i+1}.csv", index=False)
    print(f"✅ Created: employees_part_{i+1}.csv ({len(chunk_data)} records)")

print(f"\n📁 Directory: {multi_dir}")

📝 Creating multiple CSV files...
✅ Created: employees_part_1.csv (333 records)
✅ Created: employees_part_2.csv (333 records)
✅ Created: employees_part_3.csv (334 records)

📁 Directory: /opt/spark-data/raw/employees_multi


In [20]:
print("📌 Reading multiple CSV files at once")
print("="*70)

# Read all CSV files in directory
df_multi = spark.read \
    .option("header", "true") \
    .schema(employee_schema) \
    .csv(f"{multi_dir}/*.csv")

print("✅ All files loaded successfully")
print(f"   Total rows: {df_multi.count()}")
print(f"   Files read: 3")

print("\n📊 Sample from combined data:")
df_multi.show(10)

print("\n💡 Spark automatically combines all matching files!")

📌 Reading multiple CSV files at once
✅ All files loaded successfully
   Total rows: 1000
   Files read: 3

📊 Sample from combined data:
+---+-----------------+--------------------+-----------+------+---+----------+---------------+
| id|             name|               email| department|salary|age| hire_date|_corrupt_record|
+---+-----------------+--------------------+-----------+------+---+----------+---------------+
|667|      Susan Myers|kristina57@exampl...|    Finance| 71711| 31|2016-04-22|           NULL|
|668|   Gregory Murray|justinbailey@exam...|         HR| 96261| 22|2017-04-12|           NULL|
|669|    Robert Robles|angela14@example.net|    Finance| 86244| 37|2020-08-12|           NULL|
|670|  Cynthia Sanchez|terrireyes@exampl...|    Finance| 94600| 33|2018-02-01|           NULL|
|671|    Jessica Baird|   amy32@example.net|Engineering|108614| 45|2022-09-14|           NULL|
|672|Michelle Copeland|vargassteven@exam...|Engineering|108951| 56|2016-12-24|           NULL|
|673|    

---
## ☁️ PART 10: READING FROM MINIO (S3)

In [21]:
print("📤 Writing data to MinIO (S3)...")
print("="*70)

try:
    # Write Parquet to MinIO
    s3_path = f"{DATA_RAW_S3}/employees/employees.parquet"
    
    df_parquet.write \
        .mode("overwrite") \
        .parquet(s3_path)
    
    print(f"✅ Data written to MinIO: {s3_path}")
    
except Exception as e:
    print(f"❌ Failed to write to MinIO: {str(e)[:200]}")
    print("\n💡 Make sure:")
    print("   1. MinIO is running (docker-compose ps)")
    print("   2. Buckets are created (check MinIO console)")
    print("   3. Credentials are correct")

📤 Writing data to MinIO (S3)...


26/01/03 10:36:17 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


✅ Data written to MinIO: s3a://raw/employees/employees.parquet


In [22]:
print("📥 Reading data from MinIO (S3)...")
print("="*70)

try:
    # Read from MinIO
    df_from_s3 = spark.read.parquet(f"{DATA_RAW_S3}/employees/employees.parquet")
    
    print("✅ Data read from MinIO successfully")
    print(f"   Rows: {df_from_s3.count()}")
    print(f"   Columns: {len(df_from_s3.columns)}")
    
    print("\n📊 Sample data from S3:")
    df_from_s3.show(5)
    
    print("\n✅ MinIO (S3) integration working!")
    
except Exception as e:
    print(f"❌ Failed to read from MinIO: {str(e)[:200]}")

📥 Reading data from MinIO (S3)...
✅ Data read from MinIO successfully
   Rows: 1000
   Columns: 7

📊 Sample data from S3:
+---+-----------+--------------------+----------+---+-----------------+------+
|age| department|               email| hire_date| id|             name|salary|
+---+-----------+--------------------+----------+---+-----------------+------+
| 39|Engineering|donaldgarcia@exam...|2024-12-05|  1|     Allison Hill| 43278|
| 30|      Sales|jesseguzman@examp...|2017-12-29|  2|   Javier Johnson| 69256|
| 56|Engineering|  lisa02@example.net|2020-03-25|  3|Kimberly Robinson|128696|
| 49|Engineering|daviscolin@exampl...|2022-01-16|  4| Daniel Gallagher|117397|
| 27|Engineering| smiller@example.net|2022-03-11|  5|   Monica Herrera| 43905|
+---+-----------+--------------------+----------+---+-----------------+------+
only showing top 5 rows


✅ MinIO (S3) integration working!


---
## 📊 PART 11: DATA QUALITY CHECKS

In [23]:
print("🔍 DATA QUALITY CHECKS")
print("="*70)

# Use the clean parquet data
df = df_parquet

print("\n1️⃣ Basic Statistics:")
print(f"   Total rows: {df.count()}")
print(f"   Total columns: {len(df.columns)}")

print("\n2️⃣ Column Names:")
print(f"   {', '.join(df.columns)}")

print("\n3️⃣ Data Types:")
for field in df.schema.fields:
    print(f"   {field.name:<15} {str(field.dataType):<20} nullable={field.nullable}")

print("\n4️⃣ Null Counts:")
null_counts = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])
null_counts.show()

print("\n5️⃣ Numeric Statistics:")
df.select("salary", "age").describe().show()

print("\n6️⃣ Department Distribution:")
df.groupBy("department").count().orderBy("count", ascending=False).show()

🔍 DATA QUALITY CHECKS

1️⃣ Basic Statistics:
   Total rows: 1000
   Total columns: 7

2️⃣ Column Names:
   age, department, email, hire_date, id, name, salary

3️⃣ Data Types:
   age             LongType()           nullable=True
   department      StringType()         nullable=True
   email           StringType()         nullable=True
   hire_date       StringType()         nullable=True
   id              LongType()           nullable=True
   name            StringType()         nullable=True
   salary          LongType()           nullable=True

4️⃣ Null Counts:
+---+----------+-----+---------+---+----+------+
|age|department|email|hire_date| id|name|salary|
+---+----------+-----+---------+---+----+------+
|  0|         0|    0|        0|  0|   0|     0|
+---+----------+-----+---------+---+----+------+


5️⃣ Numeric Statistics:
+-------+------------------+------------------+
|summary|            salary|               age|
+-------+------------------+------------------+
|  count|    

---
## 📝 PART 12: BEST PRACTICES SUMMARY

In [24]:
print("📚 BEST PRACTICES FOR READING DATA IN SPARK")
print("="*70)

best_practices = """
✅ DO:
1. Always use explicit schema (avoid inferSchema in production)
2. Use Parquet format for production workloads
3. Use PERMISSIVE mode with _corrupt_record column to track bad data
4. Partition large datasets appropriately
5. Use MinIO/S3 for scalable storage
6. Set appropriate read options (dateFormat, nullValue, etc.)
7. Validate data quality after reading
8. Use columnar formats (Parquet, ORC) for analytics

❌ DON'T:
1. Don't use inferSchema on large datasets (slow!)
2. Don't use CSV/JSON in production (use Parquet)
3. Don't ignore corrupted records (use PERMISSIVE to track them)
4. Don't read all columns if you only need a few (use select)
5. Don't use FAILFAST without proper error handling
6. Don't store data on local filesystem in production (use S3/HDFS)

🎯 PERFORMANCE TIPS:
1. Parquet > ORC > JSON > CSV (in terms of speed)
2. Explicit schema > Schema inference (2-3x faster)
3. Predicate pushdown works best with Parquet
4. Use partitioning for large datasets
5. Coalesce/repartition after reading if needed

📊 FORMAT RECOMMENDATIONS:
• Development/Testing: CSV (easy to inspect)
• Production: Parquet (fast, compressed, schema-embedded)
• Streaming: JSON (flexible schema)
• Data Lake: Parquet with partitioning
"""

print(best_practices)

📚 BEST PRACTICES FOR READING DATA IN SPARK

✅ DO:
1. Always use explicit schema (avoid inferSchema in production)
2. Use Parquet format for production workloads
3. Use PERMISSIVE mode with _corrupt_record column to track bad data
4. Partition large datasets appropriately
5. Use MinIO/S3 for scalable storage
6. Set appropriate read options (dateFormat, nullValue, etc.)
7. Validate data quality after reading
8. Use columnar formats (Parquet, ORC) for analytics

❌ DON'T:
1. Don't use inferSchema on large datasets (slow!)
2. Don't use CSV/JSON in production (use Parquet)
3. Don't ignore corrupted records (use PERMISSIVE to track them)
4. Don't read all columns if you only need a few (use select)
5. Don't use FAILFAST without proper error handling
6. Don't store data on local filesystem in production (use S3/HDFS)

🎯 PERFORMANCE TIPS:
1. Parquet > ORC > JSON > CSV (in terms of speed)
2. Explicit schema > Schema inference (2-3x faster)
3. Predicate pushdown works best with Parquet
4. Use par

---
## 🎓 PART 13: EXERCISES

In [25]:
print("🎓 EXERCISES")
print("="*70)

exercises = """
📝 Exercise 1: Schema Definition
Define an explicit schema for a transactions dataset with:
- transaction_id (integer)
- customer_id (integer)
- amount (double)
- transaction_date (date)
- status (string)

📝 Exercise 2: Read with Options
Read a CSV file with:
- Custom delimiter (|)
- Custom null value ("N/A")
- Custom date format ("dd/MM/yyyy")
- DROPMALFORMED mode

📝 Exercise 3: Performance Test
Compare read performance of:
- CSV with inferSchema
- CSV with explicit schema
- Parquet
Measure time for reading and counting rows.

📝 Exercise 4: Corrupted Data Handling
Create a CSV with intentional errors and:
- Read with PERMISSIVE mode
- Identify corrupted records
- Save clean records to staging
- Save corrupted records to error log

📝 Exercise 5: Multi-file Reading
Create 5 CSV files with different date ranges and:
- Read all files at once
- Filter by date range
- Aggregate by month
- Write result to Parquet

📝 Exercise 6: S3 Integration
- Write data to MinIO in Parquet format
- Read it back
- Perform transformations
- Write result to different S3 bucket
"""

print(exercises)

🎓 EXERCISES

📝 Exercise 1: Schema Definition
Define an explicit schema for a transactions dataset with:
- transaction_id (integer)
- customer_id (integer)
- amount (double)
- transaction_date (date)
- status (string)

📝 Exercise 2: Read with Options
Read a CSV file with:
- Custom delimiter (|)
- Custom null value ("N/A")
- Custom date format ("dd/MM/yyyy")
- DROPMALFORMED mode

📝 Exercise 3: Performance Test
Compare read performance of:
- CSV with inferSchema
- CSV with explicit schema
- Parquet
Measure time for reading and counting rows.

📝 Exercise 4: Corrupted Data Handling
Create a CSV with intentional errors and:
- Read with PERMISSIVE mode
- Identify corrupted records
- Save clean records to staging
- Save corrupted records to error log

📝 Exercise 5: Multi-file Reading
Create 5 CSV files with different date ranges and:
- Read all files at once
- Filter by date range
- Aggregate by month
- Write result to Parquet

📝 Exercise 6: S3 Integration
- Write data to MinIO in Parquet form

---
## 🧹 CLEANUP

In [26]:
print("🧹 Cleanup (optional)")
print("="*70)
print("\nTo clean up test data, uncomment and run:")
print("""\n# import shutil
# shutil.rmtree(f"{DATA_RAW_LOCAL}/employees", ignore_errors=True)
# shutil.rmtree(f"{DATA_RAW_LOCAL}/employees_multi", ignore_errors=True)
# print("✅ Test data cleaned up")
""")

print("\n✅ Notebook completed successfully!")
print("\n📚 Next: Notebook 2 - Writing Data")

🧹 Cleanup (optional)

To clean up test data, uncomment and run:

# import shutil
# shutil.rmtree(f"{DATA_RAW_LOCAL}/employees", ignore_errors=True)
# shutil.rmtree(f"{DATA_RAW_LOCAL}/employees_multi", ignore_errors=True)
# print("✅ Test data cleaned up")


✅ Notebook completed successfully!

📚 Next: Notebook 2 - Writing Data
